# Initialize Morpheus

In [1]:
from morpheus.project.application.write import project_command_bus

# Create a Project

In [2]:
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags

user_id = UserId.new()
project_id = ProjectId.new()
name = Name.from_str('Project 1')
description = Description.from_str('Description 1')
tags = Tags.from_list(['tag1', 'tag2'])

from morpheus.project.application.write.Project import CreateProjectCommand

create_project_command = CreateProjectCommand(
  project_id=project_id,
  name=name,
  description=description,
  tags=tags,
  user_id=user_id
)

project_command_bus.dispatch(create_project_command)

# Read ProjectList

In [3]:
from morpheus.project.application.read.ProjectReader import project_reader

project_list = project_reader.get_project_summaries()
project_list

[ProjectSummary(project_id=ProjectId(value='8478ddf7-1c43-4aee-969e-af7dbdac18bc'), project_name=Name(value='Rio Primero 123'), project_description=Description(value='Rio Primero Project in Argentina'), project_tags=Tags(value=['rio primero', 'argentina']), owner_id=UserId(value='47e023c1-6939-480c-b47e-6bcae8442850'), visibility=<Visibility.PRIVATE: 'private'>, created_at=DateTime(value=datetime.datetime(2024, 5, 15, 17, 53, 42, 470799, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), updated_at=DateTime(value=datetime.datetime(2024, 5, 15, 17, 53, 42, 470799, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))))),
 ProjectSummary(project_id=ProjectId(value='e0166ed1-82ae-4f82-b154-9276aa81f1fa'), project_name=Name(value='Project 0'), project_description=Description(value='Description 0'), project_tags=Tags(value=['tag0', 'tag1']), owner_id=UserId(value='f31b40f5-309b-43f4-9f5a-fb9e5ba6ef40'), visibility=<Visibility.PRIVATE: 'private'>, created_at=DateTime(value=da

# Create model

In [4]:
from morpheus.project.types.discretization.spatial import Rotation
from morpheus.project.types.geometry import Polygon
from morpheus.project.types.Model import ModelId
from morpheus.project.application.write.Model import CreateModelCommand

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

n_cols = 100
n_rows = 50
rotation = 0.0

command = CreateModelCommand(
  project_id=project_id,
  user_id=user_id,
  model_id=ModelId.new(),
  geometry=polygon,
  n_cols=n_cols,
  n_rows=n_rows,
  rotation=Rotation.from_float(rotation),
)

project_command_bus.dispatch(command)

# Read Latest Model

In [5]:
from morpheus.project.application.read.ModelReader import ModelReader

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model

Model(model_id=ModelId(value='84c2dfe7-ba96-4aa0-955e-ecb5c5ed76fb'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[[13.922514437551428, 50.964720483303836], [13.925250781947113, 50.965228748412386], [13.925036413951403, 50.96623732041704], [13.92222441026388, 50.96629040370362], [13.922514437551428, 50.964720483303836]]], type='Polygon'), grid=Grid(origin=Point(coordinates=[13.922224410263878, 50.9662904037036], type='Point'), col_widths=[3.3689415472885593, 3.3689415472885593, 3.3689415472885598, 3.368941547288559, 3.368941547288559, 3.3689415472885607, 3.3689415472885607, 3.368941547288557, 3.368941547288557, 3.3689415472885607, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.368941547288564, 3.36894154728855, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.3689415472885713, 3.368941547288557, 3.368941547288557, 3.368941547288557, 3.368941547288557, 3.3689415472885713, 3.3689

## Update Model Geometry


In [6]:
from morpheus.project.application.write.Model import UpdateModelGeometryCommand

new_polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922, 50.965),
    (13.925, 50.965),
    (13.925, 50.966),
    (13.922, 50.966),
    (13.922, 50.965)
  ]]
)

command = UpdateModelGeometryCommand(
  project_id=project_id,
  geometry=new_polygon,
  user_id=user_id
)

project_command_bus.dispatch(command)

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model.spatial_discretization.geometry

Polygon(coordinates=[[[13.922, 50.965], [13.925, 50.965], [13.925, 50.966], [13.922, 50.966], [13.922, 50.965]]], type='Polygon')

## Update Model Grid Properties

In [7]:
from morpheus.project.application.write.Model import UpdateModelGridCommand

new_del_col_relative = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
new_del_row_relative = [0, 0.2, 0.4, 0.6, 0.8, 1]

command = UpdateModelGridCommand(
  project_id=project_id,
  user_id=user_id,
  n_cols=len(new_del_col_relative),
  n_rows=len(new_del_row_relative),
  rotation=10,
)

project_command_bus.dispatch(command)

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
print(model.spatial_discretization.geometry)
print(model.spatial_discretization.grid)
print(model.spatial_discretization.affected_cells.to_dict())

Polygon(coordinates=[[[13.922, 50.965], [13.925, 50.965], [13.925, 50.966], [13.922, 50.966], [13.922, 50.965]]], type='Polygon')
Grid(origin=Point(coordinates=[13.921728464165456, 50.96596984662515], type='Point'), col_widths=[32.68894794320857, 32.68894794320857, 32.68894794320856, 32.688947943208575, 32.6889479432086, 32.68894794320852, 32.6889479432086, 32.68894794320855, 32.68894794320863, 32.688947943208575, 32.68894794320852], total_width=359.57842737529427, row_heights=[38.67716080928221, 38.67716080928221, 38.67716080928221, 38.67716080928221, 38.677160809282185, 38.67716080928224], total_height=232.06296485569328, rotation=Rotation(value=10), length_unit=LengthUnit(unit=2))
{'type': 'raster', 'empty_value': False, 'shape': [6, 11], 'data': [[False, True, True, True, False, False, False, False, False, False, False], [False, True, True, True, True, True, True, True, True, True, True], [True, True, True, True, True, True, True, True, True, True, True], [True, True, True, True, T

## Update Model Affected Cells

In [8]:
from morpheus.project.application.write.Model import UpdateModelAffectedCellsCommand

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id=project_id)
affected_cells = model.spatial_discretization.affected_cells

assert affected_cells.is_active(col=0, row=0) == False
affected_cells.set_active(col=0, row=0)
assert affected_cells.is_active(col=0, row=0) == True

command = UpdateModelAffectedCellsCommand(
  project_id=project_id,
  affected_cells=affected_cells,
  user_id=user_id
)

project_command_bus.dispatch(command)

In [9]:
model_reader = ModelReader()
model = model_reader.get_latest_model(project_id=project_id)
affected_cells = model.spatial_discretization.affected_cells
assert affected_cells.is_active(col=0, row=0) == True

## Show Versions of Model so far

In [10]:
model_reader = ModelReader()
model_reader.get_versions(project_id=project_id)

[ModelVersion(version_id=VersionId(value='1c43d008-68a4-4632-be8d-ae90bfd1f0f3'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

## Tag Version

In [11]:
from morpheus.project.application.write.Model import CreateModelVersionCommand
from morpheus.project.types.ModelVersion import VersionTag, VersionDescription

command = CreateModelVersionCommand(
  project_id=project_id,
  version_tag=VersionTag('v1.0.0'),
  version_description=VersionDescription('Version 1.0.0'),
  user_id=user_id
)

project_command_bus.dispatch(command)

In [12]:
model_reader.get_versions(project_id=project_id)

[ModelVersion(version_id=VersionId(value='30f5c538-22ca-40db-953c-9022c00c9bcb'), tag=VersionTag(value='v1.0.0'), description=VersionDescription(value='Version 1.0.0')),
 ModelVersion(version_id=VersionId(value='1c43d008-68a4-4632-be8d-ae90bfd1f0f3'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

In [13]:
version = model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version

ModelVersion(version_id=VersionId(value='30f5c538-22ca-40db-953c-9022c00c9bcb'), tag=VersionTag(value='v1.0.0'), description=VersionDescription(value='Version 1.0.0'))

In [14]:
from morpheus.project.application.write.Model import UpdateModelVersionDescriptionCommand

command = UpdateModelVersionDescriptionCommand(
  project_id=project_id,
  version_id=version.version_id,
  version_description=VersionDescription('Version 1.0.0 - Updated'),
  user_id=user_id
)

project_command_bus.dispatch(command)

In [15]:
version = model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version.description.to_str()

'Version 1.0.0 - Updated'

# Add well boundary

In [16]:
from morpheus.project.application.write.Model.AddModelBoundary import AddModelBoundaryCommandPayload
from morpheus.project.application.write.Model import AddModelBoundaryCommand

payload: AddModelBoundaryCommandPayload = {
  'project_id': project_id.to_str(),
  'model_id': model.model_id.to_str(),
  'boundary_type': 'well',
  'boundary_geometry': {
    'type': 'Point',
    'coordinates': [13.923, 50.965]
  },
}

project_command_bus.dispatch(AddModelBoundaryCommand.from_payload(user_id=user_id, payload=payload))

# Read Model Boundaries

In [17]:
model = model_reader.get_latest_model(project_id=project_id)
model.boundaries

BoundaryCollection(boundaries=[WellBoundary(boundary_id=BoundaryId(value='8f44097c-9a72-48e6-b9f5-e0a6041d9afa'), type=BoundaryType(type='well'), name=BoundaryName(value='new well boundary'), tags=BoundaryTags(value=[]), geometry=Point(coordinates=[13.923, 50.965], type='Point'), affected_cells=ActiveCells(shape=[6, 11], data=[ActiveCell(col=3, row=5)]), affected_layers=[LayerId(value='375d0b31-7308-4a25-845f-5538c769d8b6')], observations=[WellObservation(observation_id=ObservationId(value='3b077460-85ec-4e95-9f2c-ab85fe77989f'), observation_name=ObservationName(value='Observation Point 1'), geometry=Point(coordinates=[13.923, 50.965], type='Point'), data=[WellRawDataItem(date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)), pumping_rate=PumpingRate(value=0.0))])], enabled=True)])

# Remove Well Boundary

In [18]:
from morpheus.project.application.write.Model.RemoveModelBoundary import RemoveModelBoundaryCommandPayload, RemoveModelBoundaryCommand

boundary_id = model.boundaries[0].boundary_id

payload: RemoveModelBoundaryCommandPayload = {
  'project_id': project_id.to_str(),
  'model_id': model.model_id.to_str(),
  'boundary_id': boundary_id.to_str(),
}

project_command_bus.dispatch(RemoveModelBoundaryCommand.from_payload(user_id=user_id, payload=payload))

In [19]:
model = model_reader.get_latest_model(project_id=project_id)
model.boundaries

BoundaryCollection(boundaries=[])